# 🚀 cuOpt Examples Launcher

Welcome to the cuOpt Examples Repository! This notebook helps you:
- 📥 Clone or update the latest cuopt-examples repository
- 🌿 Select and switch between different branches
- 📂 Interactively browse all available notebooks
- 🔗 Launch notebooks directly in Google Colab

> **💡 Note:** All implementation details are in `cuopt_launcher_utils.py`. This notebook stays clean and focused on the user interface!  
> When opened in Colab, the utility file is automatically downloaded from GitHub.

---

## 🎯 Quick Start

1. **First Time**: Run all cells (Cell → Run All)
2. **Hide Code** (Optional): 
   - In **Jupyter Lab/Notebook**: Right-click on code cells → "Collapse Selected Code"
   - In **VS Code**: Click the chevron (^) next to the cell number to collapse
   - In **Google Colab**: Click the [ ] bracket to the left of code cells
3. **Scroll down** to see the interactive interface!

Most code cells run silently in the background. Only the setup and UI cells show output.

---


## ⚙️ Setup (these cells run silently - you can collapse/hide them)


In [ ]:
%%capture --no-display
# Install required packages silently
%pip install ipywidgets gitpython -q


In [ ]:
# ============================================================================
# IMPORT UTILITIES - All implementation details are in cuopt_launcher_utils.py
# ============================================================================

# Download utility file if not present (for Colab compatibility)
from pathlib import Path

if not Path('cuopt_launcher_utils.py').exists():
    print("📥 Downloading utility file from GitHub...")
    import urllib.request
    
    url = "https://raw.githubusercontent.com/NVIDIA/cuopt-examples/cuopt_examples_launcher/cuopt_launcher_utils.py"
    urllib.request.urlretrieve(url, 'cuopt_launcher_utils.py')
    print("✅ Utility file downloaded successfully!")

# Now import from the utility file
from IPython.display import display, Markdown
from ipywidgets import Button, HBox, Output, Layout

# Import all utility functions
from cuopt_launcher_utils import (
    clone_or_update_repo,
    display_interactive_ui,
    run_command,
    REPO_PATH
)

print("✅ All utilities loaded successfully!")


In [ ]:
# Clone or update the repository
clone_or_update_repo()

print("\n" + "="*60)
print("✅ Setup complete! Scroll down for interactive controls.")
print("="*60)


---

## 🎯 Interactive Controls

All setup is complete! The cell below displays your interactive interface.


### Usage Tips

1. **Branch Selection**: Different branches may contain different versions or experimental features
2. **Colab Execution - IMPORTANT**: 
   - Click "Open in Colab" to run notebooks in Google Colab with free GPU access
   - **⚠️ For notebooks that use data files**: Click "📋 Copy Setup Code" button first
   - Paste the setup code in the first cell of your Colab notebook and run it
   - This ensures all data files, images, and dependencies are available
3. **Local Execution**: You can also run notebooks locally if you have the required dependencies
4. **Requirements**: Check `requirements.txt` in the repository for necessary Python packages

### Working with Colab - Step by Step

If a notebook requires data files (most do), follow these steps:

1. Click **"📋 Copy Setup Code"** for the notebook you want to run
2. Click **"Open in Colab"** to open the notebook
3. In Colab, insert a new cell at the top (Code > Insert code cell)
4. Paste the setup code and run it (this clones the repo and sets up the environment)
5. Now run the rest of the notebook - all files will be accessible!

### Links

- [GitHub Repository](https://github.com/NVIDIA/cuopt-examples)
- [NVIDIA cuOpt Documentation](https://docs.nvidia.com/cuopt/)
- [Report Issues](https://github.com/NVIDIA/cuopt-examples/issues)

---

<div style='text-align: center; padding: 20px; background-color: #f0f0f0; border-radius: 10px;'>
    <strong>Made with ❤️ for the NVIDIA cuOpt Community</strong>
</div>


In [ ]:
# ============================================================================
# INTERACTIVE UI - RUN THIS CELL TO SEE THE INTERFACE
# ============================================================================

# Display the complete interactive interface
display_interactive_ui()
